# Imports

In [1]:
# General imports
import os
import sys
import random 
import functools

import cv2
import piq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm
import matplotlib.pyplot as plt 
import albumentations as albu
import albumentations.pytorch as albu_pt

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

# Fix to be able to import python modules inside a notebook
os.chdir('..')

# Useful extensions
# %load_ext watermark
# %watermark -v -n -m -p numpy,torch,albumentations,photosynthesis_metrics

# %load_ext autoreload
# %autoreload 2

# Nice plot formating
%matplotlib inline

In [3]:
!ls

configs  logs	   models     old_logs	 reports	   src	  train.py
data	 Makefile  notebooks  README.md  requirements.txt  tests  WORKPLAN.md


Training GAN's experiments:

GAN runner
GAN callback to support custom metrics

GAN model (from Skoltech DL course)



In [9]:
import src.trainer.callbacks as clbs

phases = {
  "generator": [
    {"ep":[0,8],  "lr":[0,0.1], "mom":0.9, },
    {"ep":[8,24], "lr":[0.1, 0.01], "mode":"cos"},
    {'ep':[24, 30], "lr": 0.001}],
  "discriminator": [
    {"ep":[0,8],  "lr":[0,0.1], "mom":0.9, },
    {"ep":[8,24], "lr":[0.1, 0.01], "mode":"cos"},
    {'ep':[24, 30], "lr": 0.001}],
}

sheduler = clbs.PhasesScheduler(phases)

# sheduler.phases, sheduler.phase, sheduler.tot_epochs

sheduler._get_lr_mom()

TypeError: _get_lr_mom() missing 2 required positional arguments: 'phase' and 'batch_curr'

In [31]:
from src.data import crop_patches

target = torch.rand(4, 3, 128, 128)
target_patches = crop_patches(target, size=96, stride=32)
print(target_patches.shape)

dataset = torch.utils.data.TensorDataset(target_patches)
print(dataset[0][0].shape)

torch.Size([16, 3, 96, 96])
torch.Size([3, 96, 96])


In [26]:
class PSNR(torch.nn.modules.loss._Loss):
    def __init(self, data_range=1.0, reduction='mean', convert_to_greyscale: bool = False):
        super().__init__()
        
        self.metric = functools.partial(
            piq.psnr, data_range=data_range, reduction=reduction, convert_to_greyscale=convert_to_greyscale)

    def forward(self, prediction: torch.Tensor, target: torch.Tensor):
        self.metric(prediction, target)


kwargs = {'data_range': 1.0, 'convert_to_greyscale': False},
PSNR(kwargs)

PSNR()

In [24]:
issubclass(piq.GMSDLoss, torch.nn.modules.loss._Loss)
issubclass(piq.fsim, torch.nn.modules.loss._Loss)

TypeError: issubclass() arg 1 must be a class

In [45]:
a = torch.rand(2, 3, 4, 4, 2)
res = torch.split(a, split_size_or_sections=1, dim=4)
res[1].shape

torch.Size([2, 3, 4, 4, 1])

In [15]:
piq.multi_scale_ssim?

In [13]:
prediction = torch.rand(4, 3, 128, 128)
piq.MultiScaleGMSDLoss()(prediction, prediction)

tensor(0.)

In [4]:
piq.ContentLoss?

In [2]:
a = "model_{ep}_{metric:.2f}.chpn"
a.format(ep=3, metric=0.234556436)

'model_3_0.23.chpn'

In [12]:
kwargs = {
    "feature_extractor": 'vgg19',
    "layers": ['conv1_2', 'conv2_2', 'conv3_4', 'conv4_4', 'conv5_4'],
    "weights": [0.2, 0.2, 0.2, 0.2, 0.2],
    "normalize_features": True,
    "reduction": 'none',
}

kwargs.pop("feature_extractor")
kwargs

{'layers': ['conv1_2', 'conv2_2', 'conv3_4', 'conv4_4', 'conv5_4'],
 'weights': [0.2, 0.2, 0.2, 0.2, 0.2],
 'normalize_features': True,
 'reduction': 'none'}

In [9]:
name = "content"
str.includes('good')
# if name in ["content", "style"]:
#     print("Yes")

AttributeError: type object 'str' has no attribute 'includes'

In [19]:
# piq.SSIMLoss?
eval('["ssim", {"kernel_size" : 7}, "ms_ssim", {"kernel_size" : 7}, "psnr", {}, ]')

['ssim', {'kernel_size': 7}, 'ms_ssim', {'kernel_size': 7}, 'psnr', {}]

In [13]:
class A:
    _folder = "A"
    def forward(self, i):
        return self._folder + str(i)

metrics = [A(), A(), A(), A()]
metric_names = ["A", "a", "AA", "aa"]

for metric, name in zip(metrics, metric_names):
    metric.name = name
    

[m.name=n for (m, n) in zip(metrics, metric_names)]
metrics[0].name

SyntaxError: invalid syntax (<ipython-input-13-762d806b0f6f>, line 13)

In [ ]:
        # DummyAverageMeter is added to the end, so just delete last part
        counter = 0
        for i, metric in enumerate(self.state.metric_meters):
            if isinstance(metric, AverageMeter):
                metric.reset()
            else:
                counter += 1
        self.state.metric_meters = self.state.metric_meters[:-counter]

In [67]:
reduction = 'none'
f = functools.partial(piq.vif_p, reduction=reduction)
f.name = "test"
print(f, f.name)

functools.partial(<function vif_p at 0x7f59f1780d08>, reduction='none') test


In [62]:
class A():
    def reset(self):
        print("A reseted")


class B():
    def reset(self):
        print("B reseted")

class State:
    def __init__(self, metric_meters):
        self.metric_meters = metric_meters

metric_meters = [A(), A(), B(), B(), B(), A()]
print(metric_meters)
state = State(metric_meters)

state.metric_meters = [m for m in state.metric_meters if isinstance(m, A)]
for metric in state.metric_meters:
    metric.reset()

# omelist = [x for x in somelist if not determine(x)]

# for i, metric in enumerate(state.metric_meters):
#     if isinstance(metric, A):
#         metric.reset()
#     elif isinstance(metric, B):
#         print("Deleting B")
#         del state.metric_meters[i]
print(state.metric_meters)
        
    

[<__main__.A object at 0x7f59d08592e8>, <__main__.A object at 0x7f59d0859860>, <__main__.B object at 0x7f59d08597f0>, <__main__.B object at 0x7f59d08598d0>, <__main__.B object at 0x7f59d0859898>, <__main__.A object at 0x7f59d0859828>]
A reseted
A reseted
A reseted
[<__main__.A object at 0x7f59d08592e8>, <__main__.A object at 0x7f59d0859860>, <__main__.A object at 0x7f59d0859828>]


In [51]:
import functools
METRIC_FROM_NAME = {
    # Full Reference
    "psnr": functools.partial(
        piq.psnr),
}

def get_metric(name, reduction='none'):
    metric = METRIC_FROM_NAME[name]
    return functools.partial(metric, reduction=reduction)
    
prediction = torch.rand(4, 3, 128, 128)
target = torch.rand(4, 3, 128, 128)

get_metric("psnr")(prediction, target)

tensor(7.7687)

In [55]:
a = [piq.ContentLoss(), piq.StyleLoss(), piq.LPIPS(), piq.DISTS(), piq.GMSDLoss()]

In [ ]:
[07-24 20:48] - Best loss: 0.0394
[07-24 20:48] - Best psnr: 25.0806
[07-24 20:48] - Best ssim: 0.7546
[07-24 20:48] - Best ms-ssim: 0.9332
[07-24 20:48] - Best gmsd: 0.0846
[07-24 20:48] - Best ms-gmsd: 0.0816
[07-24 20:48] - Best ms-gmsdc: 0.0681
[07-24 20:48] - Best fsim: 0.8802
[07-24 20:48] - Best fsimc: 0.8744
[07-24 20:48] - Best vsi: 0.9544
[07-24 20:48] - Best mdsi: 0.4146
[07-24 20:48] - Best vifp: 0.9085
[07-24 20:48] - Best content_vgg16: 0.2474
[07-24 20:48] - Best content_vgg19: 0.2679
[07-24 20:48] - Best content_vgg16_ap: 0.2176
[07-24 20:48] - Best content_vgg19_ap: 0.2257
[07-24 20:48] - Best style_vgg16: 9222874.0769
[07-24 20:48] - Best style_vgg19: 7961090.4615
[07-24 20:48] - Best lpips: 0.2275
[07-24 20:48] - Best dists: 0.1868
[07-24 20:48] - Best brisque: 37.7652

In [56]:
for l in a:
    print(isinstance(l, piq.ContentLoss))

True
True
True
True
False


In [5]:
class A:
    _folder = "A"
    def forward(self, i):
        return self._folder + str(i)
    
class B(A):
    _folder = "B"
    def forward(self, i):
        res = super().forward(i)
        return res

a = A()
b = B()
a.forward(3), b.forward(4)

('A3', 'B4')

tensor(120.5809)

In [4]:
a = [1, 2]
b = [3, 4]
c = [5, 6, 4]
len(a) == len(b) == len(c) == 2

False

In [21]:
"|".join(["a", "b"])


'a|b'

In [14]:
from pytorch_tools.utils.misc import listify
PHASES = {
    "generator": 
        [{"ep":[0,8],  "lr":[0,0.1], "mom":0.9, },
        {"ep":[8,24], "lr":[0.1, 0.01], "mode":"cos"},
        {'ep':[24, 30], "lr": 0.001}],
    "discriminator": 
        [{"ep":[0,8],  "lr":[0,0.1], "mom":0.9, },
        {"ep":[8,24], "lr":[0.1, 0.01], "mode":"cos"},
        {'ep':[24, 30], "lr": 0.001}],
}

def _format_phase(phase):
    phase["ep"] = listify(phase["ep"])
    phase["lr"] = listify(phase["lr"])
    phase["mom"] = listify(phase.get("mom", None))  # optional
    if len(phase["lr"]) == 2 or len(phase["mom"]) == 2:
        phase["mode"] = phase.get("mode", "linear")
        assert len(phase["ep"]) == 2, "Linear learning rates must contain end epoch"
    return phase

# [_format_phase(p) for p in PHASES["generator"]]
list(map(_format_phase, PHASES["generator"]))


[[{'ep': [0, 8], 'lr': [0, 0.1], 'mom': [0.9], 'mode': 'linear'},
  {'ep': [8, 24], 'lr': [0.1, 0.01], 'mode': 'cos', 'mom': []},
  {'ep': [24, 30], 'lr': [0.001], 'mom': []}]]

In [15]:
feature = torch.rand(10, 256)
prediction_features = [feature, feature, feature]
target_features = [feature, feature]

prediction_features = torch.cat(prediction_features, dim=0)
target_features = torch.cat(target_features, dim=0)

print(prediction_features.shape, target_features.shape)

torch.Size([30, 256]) torch.Size([20, 256])


In [16]:
NAMES = ['a', 'b']

c = 
c

{'a': None, 'b': None}